# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
import csv
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk import RegexpParser
from nltk.tree import Tree
import requests
from bs4 import BeautifulSoup
import spacy
from collections import Counter

In [ ]:
movie_id = "tt9426210"  # John Wick: Chapter 4
base_url = f"https://www.imdb.com/title/{movie_id}/reviews"
n = 1000
reviews = []


page_num = 1
while len(reviews) < n:
    page_url = f"{base_url}?start={page_num}"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, "html.parser")


    review_elements = soup.find_all("div", class_="review-container")


    for review_element in review_elements:
        review_content = review_element.find("div", class_="text show-more__control")
        if review_content is not None:
            review_text = review_content.get_text()
            reviews.append(review_text.strip())

    page_num += 25


df = pd.DataFrame({"Review" : reviews})

df.to_csv("review.csv")


In [ ]:
review_df = pd.read_csv("review.csv")
review_df.drop(["Unnamed: 0"] , axis = 1 , inplace = True)
review_df

,Review
0,I can't help but suspect that writer/director ...
1,Weathering With You is an anime fantasy film p...
2,One of the best experiences I ever had! So gla...
3,Hina is literally sunshine. She walks through ...
4,I'm interested in this movie because of shinka...
...,...
995,Weathering With You is an incredible anime tha...
996,Weathering with you is yet another great to lo...
997,"The art , the animation , the soundtrack , eve..."
998,Almost as good as your name. Great story beaut...


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


stop_words = set(stopwords.words('english'))


clean_reviews = []
for review in review_df["Review"].values:

    review = review.lower()

    review = re.sub(r'\W', ' ', review)

    review = re.sub(r'\d', ' ', review)

    tokens = nltk.word_tokenize(review)

    clean_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens if token not in stop_words]
    clean_review = ' '.join(clean_tokens)
    clean_reviews.append(clean_review)


df["Cleaned Review"] = clean_review

df.to_csv("review.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
review_df = pd.read_csv("review.csv")
review_df.drop(["Unnamed: 0"] , axis = 1 , inplace = True)
review_df

,Review,Cleaned Review
0,I can't help but suspect that writer/director ...,last movi makoto shinkai film name devastating...
1,Weathering With You is an anime fantasy film p...,last movi makoto shinkai film name devastating...
2,One of the best experiences I ever had! So gla...,last movi makoto shinkai film name devastating...
3,Hina is literally sunshine. She walks through ...,last movi makoto shinkai film name devastating...
4,I'm interested in this movie because of shinka...,last movi makoto shinkai film name devastating...
...,...,...
995,Weathering With You is an incredible anime tha...,last movi makoto shinkai film name devastating...
996,Weathering with you is yet another great to lo...,last movi makoto shinkai film name devastating...
997,"The art , the animation , the soundtrack , eve...",last movi makoto shinkai film name devastating...
998,Almost as good as your name. Great story beaut...,last movi makoto shinkai film name devastating...


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

nlp = spacy.load('en_core_web_sm')


pos_counts = Counter()


constituency_trees = []


named_entities = []


for review in review_df["Cleaned Review"].values:

    tokens = nltk.word_tokenize(review)
    pos_tags = pos_tag(tokens)


    pos_counts.update(tag for word, tag in pos_tags if tag.startswith(('N', 'V', 'J', 'R')))


    pattern = 'NP: {<DT>?<JJ>*<NN>}'
    parser = RegexpParser(pattern)
    constituency_tree = parser.parse(pos_tags)
    constituency_trees.append(constituency_tree)


    doc = nlp(review)
    named_entities.extend(ent.label_ for ent in doc.ents)


print('Parts of Speech counts:')
for tag, count in pos_counts.items():
    print(f'{tag}: {count}')


print('Example constituency parsing tree:')
print(constituency_trees[0])


print('Named entity counts:')
print(Counter(named_entities))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Parts of Speech counts:
JJ: 4000
NN: 12000
VB: 1000
VBP: 1000

Example constituency parsing tree:
(S
  (NP last/JJ movi/NN)
  (NP makoto/NN)
  (NP shinkai/NN)
  (NP film/NN)
  (NP name/NN)
  devastatingli/VB
  (NP amaz/JJ time/NN)
  (NP mmm/NN)
  (NP underplot/NN)
  until/IN
  (NP end/JJ movi/NN)
  (NP mean/NN)
  anyth/VBP
  (NP obvious/JJ redund/NN)
  (NP scene/NN))

Named entity counts:
Counter()


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

- I felt the assignmnet was a little bit hard , as impelemeting as Data extraction from different website and cleaning them is not easy task